# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
import requests
from dotenv import load_dotenv

In [ ]:
#| export
def search_wikidata_properties_by_name_and_aliases(search_term):
    """
    Search Wikidata properties by their name or aliases using a SPARQL query.
    
    This function queries the Wikidata SPARQL endpoint to find properties that match a given search term by their name or aliases.
    It returns a limited set of results that include the property ID, label, description, and aliases.
    
    Parameters:
    - search_term: A string containing the term to search for in the property names and aliases.
    
    Returns:
    A list of found properties with their ID, label, description, and aliases. If no properties are found or an error occurs, a message is printed.
    """
    
    # Define the SPARQL query. The query searches for properties matching the search term in their labels or aliases.
    query = f'''
    SELECT DISTINCT ?property ?propertyLabel ?propertyDescription (GROUP_CONCAT(DISTINCT ?alias;separator=", ") AS ?aliases)
    WHERE {{
      ?property a wikibase:Property .
      {{?property rdfs:label ?label FILTER (LANG(?label) = "en").}}
      OPTIONAL {{?property skos:altLabel ?alias FILTER (LANG(?alias) = "en").}}
      FILTER(CONTAINS(LCASE(?label), LCASE("{search_term}")) || CONTAINS(LCASE(?alias), LCASE("{search_term}"))).
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    GROUP BY ?property ?propertyLabel ?propertyDescription
    LIMIT 5
    '''

    # Set up the request to the Wikidata SPARQL endpoint.
    url = 'https://query.wikidata.org/sparql'
    headers = {'User-Agent': 'Mozilla/5.0', 'Accept': 'application/sparql-results+json'}
    
    # Perform the GET request with the query.
    response = requests.get(url, headers=headers, params={'query': query, 'format': 'json'})
    
    # Process the response.
    if response.status_code == 200:
        # Parse the JSON response and extract the relevant data.
        results = response.json().get('results', {}).get('bindings', [])
        for result in results:
            # Extract and print details for each property.
            print(f"Property ID: {result.get('property', {}).get('value', 'N/A').split('/')[-1]}")  # Extracting ID from URI
            print(f"Label: {result.get('propertyLabel', {}).get('value', 'N/A')}")
            print(f"Description: {result.get('propertyDescription', {}).get('value', 'N/A')}")
            aliases = result.get('aliases', {}).get('value', 'N/A')
            print(f"Aliases: {aliases if aliases else 'None'}")
            print('---')
    else:
        # Print an error message if the request was unsuccessful.
        print("Failed to retrieve data")


Example usage of the `search_wikidata_properties_by_name_and_aliases` function

In [ ]:
search_term = "used for"  # Example search term, to find properties related to "population"
search_wikidata_properties_by_name_and_aliases(search_term)

Property ID: P2176
Label: drug or therapy used for treatment
Description: drug, procedure, or therapy that can be used to treat a medical condition
Aliases: used for treatment, treated by, prescribed drug, cure, prescription, treatment
---
Property ID: P366
Label: has use
Description: main use of the subject (includes current and former usage)
Aliases: can be used for, used for
---
Property ID: P4970
Label: alternative name
Description: qualifier for alternative name(s), given for a subject in a database entry, or preserved in references (even these are no longer the preferred name)
Aliases: used for term, used for, unused form
---
Property ID: P642
Label: of
Description: (This property is being deprecated, see details at WD:P642) qualifier stating that a statement applies within the scope of a particular item
Aliases: used for
---
Property ID: P6530
Label: has role in modeling
Description: relation between a research artefact and an entity that is used to study, in virtue of its repli

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()